# Git commands


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/My Drive/Colab Notebooks/Master_course_Homeworks/
# تنظیم ایمیل و نام (اگر سشن جدید است)
!git config --global user.email "arshia.mokhlesi@gmail.com"
!git config --global user.name "arshia.mkh"

# ساخت برنچ برای تمرین جدید
# !git checkout -b hw6-IR

/content/drive/My Drive/Colab Notebooks/Master_course_Homeworks


In [6]:
!git add .
!git commit -m "add a code only to show the code in github\n complete LLM as judge part 2 section 6\n add report of home work "
!git push origin hw6-IR

[main ec42c1b] add a code only to show the code in github\n complete LLM as judge part 2 section 6\n add report of home work
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Intelligence Information Retrieval/HW6/Conversational_Search_System.ipynb (66%)
Everything up-to-date


In [7]:
!git checkout main
!git pull origin main
!git merge hw6-IR
!git push origin main

M	Intelligence Information Retrieval/HW6/Conversational_Search_System.ipynb
Already on 'main'
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)
From https://github.com/ArsHia-cdMstr/Master_course_Homeworks
 * branch            main       -> FETCH_HEAD
Already up to date.
Already up to date.
Enumerating objects: 25, done.
Counting objects: 100% (25/25), done.
Delta compression using up to 2 threads
Compressing objects: 100% (20/20), done.
Writing objects: 100% (20/20), 273.80 KiB | 3.04 MiB/s, done.
Total 20 (delta 12), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (12/12), completed with 3 local objects.
To https://github.com/ArsHia-cdMstr/Master_course_Homeworks.git
   dd4ef76..ec42c1b  main -> main
